# Brainn Tumor Classification

# Import Required Packages

In [1]:
import os
os.chdir("../")
%pwd

'd:\\python-projects\\brain-tumor-classification'

In [2]:
from pathlib import Path
from dataclasses import dataclass
from src.utils.common import read_yaml, create_directories
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.logging import logger


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Data Ingestion

In [22]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_data_dir: Path

The *DataIngestionConfig Class* store two important paths for data processing, and the "frozen" part ensures these paths can't be accidentally modified later.

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),  
            source_data_dir=Path(config.source_data_dir)
        )

        return data_ingestion_config

*ConfigurationManager*  is a class that helps to manage and organize your project's settings.

In [27]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def initiate_data_ingestion(self):
        logger.info("Checking Data Directory is exsisting")
        if not self.config.source_data_dir.exists():
            raise FileNotFoundError(f"Data directory not found at {self.config.source_data_dir}")
        logger.info("Data directory found and ready to use")
        return self.config.source_data_dir

*DataIngestion* class is responsible for handling the data ingestion process, meaning it helps to check if the data directory exists and can be used for further processing.

In [28]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
except Exception as e:
    raise e

[2025-05-05 15:49:23,304] [14] [tumorClassifierLogger] - INFO - YAML file loaded successfully: config\config.yaml
[2025-05-05 15:49:23,307] [14] [tumorClassifierLogger] - INFO - YAML file loaded successfully: params.yaml
[2025-05-05 15:49:23,308] [27] [tumorClassifierLogger] - INFO - Created directory at: artifacts
[2025-05-05 15:49:23,309] [27] [tumorClassifierLogger] - INFO - Created directory at: artifacts\data_ingestion
[2025-05-05 15:49:23,310] [6] [tumorClassifierLogger] - INFO - Checking Data Directory is exsisting
[2025-05-05 15:49:23,311] [9] [tumorClassifierLogger] - INFO - Data directory found and ready to use
